# Sampling random pure free fermion states

1. I recommend a clean `venv` to reproduce the results below.

2. Start with `git clone https://github.com/keisuke-akira/Hoffman-Decomposition-and-the-Matchgate-Group.git`

3. Then install `free-fermion-lib`

### Imports

In [1]:
# I've used `git clone https://github.com/keisuke-akira/Hoffman-Decomposition-and-the-Matchgate-Group.git` to keep a local copy of the functions below.
import sys, pathlib
repo_dir = pathlib.Path("Hoffman-Decomposition-and-the-Matchgate-Group").resolve()
sys.path.append(str(repo_dir))

# import everything for now
from HoffmanDecomposition import *
from MatchgateDecomposition import *

### Test Matchgate conversions

In [2]:
# quick test that the Matchgate library is working well.
import numpy as np
from scipy.stats import special_ortho_group

# Number of sites
N = 3
# Generate a random rotation over the 2N Majoranas
T = special_ortho_group.rvs(2*N)

# Convert this rotation to a list of matchgates
mg_list = RotationToMatchgates(np.matrix(T))

maj_ops = FindMajoranaOperators(N)

# Check that the two match by converting back
T_back = MatchgatesToRotation(mg_list, majoranaList=maj_ops)
print("max error =", np.max(np.abs(np.asarray(T_back - T))))

max error = 1.4432941020150352e-15


In [3]:
# testing that the `RotationToMatchgates` generates a list of two-qubit matchgates. Uses `is_matchgate` function
import ff
from ff import is_matchgate
results = []
for (i, j), U2 in mg_list:
    U = np.asarray(U2, dtype=complex)  # convert numpy.matrix -> ndarray
    ok = is_matchgate(U)
    results.append((i, j, bool(ok)))

# ensure all gates passed
all_ok = all(ok for _, _, ok in results)
print("All two-qubit gates in the list are matchgates:", all_ok)
assert all_ok, "Some entries failed the matchgate check!"

All two-qubit gates in the list are matchgates: True


### Test `free-fermion-lib` works well

In [4]:
# Generate Jordan-Wigner operators for 3 sites
n_sites = 3
alphas = ff.jordan_wigner_alphas(n_sites)

# Generate a Gaussian state
rho = ff.random_FF_state(n_sites)
# print(rho)

# Compute correlation matrix
gamma = ff.compute_2corr_matrix(rho, n_sites, alphas)
# print(gamma)

# Compute pfaffian of a skew-symmetric matrix
skew_matrix = np.array([[0, 1, -2], [-1, 0, 3], [2, -3, 0]])
pfaffian_value = ff.pf(skew_matrix)
print(pfaffian_value)

0


## Using random matchgates to prepare a random state

In [5]:
from scipy.linalg import logm, expm

n_sites = 5  # number of modes/qubits

# 1) Haar-random Majorana rotation T in SO(2n)
T = special_ortho_group.rvs(2*n_sites)

# 2) Build the quadratic generator on Hilbert space using James' Majoranas
gam = ff.jordan_wigner_majoranas(n_sites)           # list [γ_0, ..., γ_{2n-1}], each 2^n x 2^n (Hermitian)
A = logm(T)                  # A in so(2n) (skew-symmetric)
A = np.real_if_close(A, tol=1e-10)
A = 0.5 * (A - A.T)          # enforce skew symmetry numerically

# K = (1/4) \sum_{μ<ν} A_{μν} γ_μ γ_ν  (anti-Hermitian)
d = gam[0].shape[0]
K = np.zeros((d, d), dtype=complex)
for mu in range(2*n_sites):
    for nu in range(mu+1, 2*n_sites):
        a = A[mu, nu]
        if abs(a) > 1e-14:
            K += (a/4.0) * (gam[mu] @ gam[nu])
K = 0.5*(K - K.conj().T)     # clean up numerical anti-Hermiticity

# 3) Exponentiate to get the Hilbert-space unitary and apply to |0...0>
U = expm(K)                  # Gaussian unitary implementing T
psi0 = np.zeros((d, 1), dtype=complex); psi0[0, 0] = 1.0
psi = U @ psi0               # <-- this is T applied to |0>^{\otimes n}

# (optional) sanity checks
import numpy.linalg as npl
print("‖ψ‖ =", float(npl.norm(psi)))
print("first few amps:", psi[:8, 0])

‖ψ‖ = 1.0
first few amps: [0.86413207+0.06956603j 0.        +0.j         0.        +0.j
 0.04709399-0.17660749j 0.        +0.j         0.12439468-0.11756247j
 0.21841512+0.00396097j 0.        +0.j        ]


In [7]:
# James' JW Majoranas (δ convention: {γ,γ} = I); different from mine!
majoranas = ff.jordan_wigner_majoranas(n_sites)
psi_dag = psi.conj().T

# Majorana covariance: G_{μν} = i * < [γ_μ, γ_ν] >
G = np.zeros((2*n_sites, 2*n_sites), dtype=float)
for mu in range(2*n_sites):
    for nu in range(2*n_sites):
        comm = majoranas[mu] @ majoranas[nu] - majoranas[nu] @ majoranas[mu]
        val = (psi_dag @ (comm @ psi)).item()   # scalar
        G[mu, nu] = (1j * val).real             # should be real up to fp noise

# Gaussianity / purity checks
antisym_err = npl.norm(G + G.T, ord='fro')              # ~ 0
purity_err  = npl.norm(G @ G + np.eye(2*n_sites), ord='fro')  # ~ 0 for pure Gaussian

print("Majorana covariance antisymmetry ‖G + G^T‖_F =", antisym_err)
print("Gaussian purity check            ‖G^2 + I‖_F =", purity_err)

Majorana covariance antisymmetry ‖G + G^T‖_F = 0.0
Gaussian purity check            ‖G^2 + I‖_F = 1.2762460033366384e-15


# Testing the 1-design nature of random FF states

In [12]:
n_sites = 6  # number of modes/qubits

psi=random_FF_state(n_sites,pure=True)
np.trace(psi.conj().T @ psi)

np.complex128(0.9999999999999951+0j)

In [15]:
import numpy as np
import numpy.linalg as npl

def parity_projectors(n: int):
    Z = np.array([[1, 0],[0, -1]], dtype=complex)
    P = np.array([[1]], dtype=complex)
    for _ in range(n):
        P = np.kron(P, Z)                 # parity operator Π_z = ⊗_j Z_j
    Id = np.eye(2**n, dtype=complex)
    P_even = (Id + P) / 2.0
    P_odd  = (Id - P) / 2.0
    return P_even, P_odd

def test_state_1_design_even_sector(n=6, M=300, seed0=123):
    d = 2**n
    rng = np.random.default_rng(seed0)
    rho_bar = np.zeros((d, d), dtype=complex)

    # Monte Carlo average of |psi><psi|
    for _ in range(M):
        psi = random_FF_state(n, pure=True, seed=int(rng.integers(1<<31)))
        psi = np.asarray(psi).reshape(d, 1)     # ensure (d,1)
        rho_bar += psi @ psi.conj().T
    rho_bar /= M

    # Even-parity maximally mixed target
    P_even, P_odd = parity_projectors(n)
    target = P_even / (2**(n-1))

    # Diagnostics
    tr = np.trace(rho_bar).real
    even_w = np.trace(P_even @ rho_bar).real
    odd_w  = np.trace(P_odd  @ rho_bar).real
    fro_err = npl.norm(rho_bar - target, ord='fro')

    return {
        "n": n, "samples": M,
        "trace(rhō)": float(tr),
        "weight_even": float(even_w),
        "weight_odd": float(odd_w),
        "fro_error_vs_even_max_mixed": float(fro_err),
    }

# 300 samples
res = test_state_1_design_even_sector(n=6, M=300, seed0=123)
print(res)

# 3000 samples
res = test_state_1_design_even_sector(n=6, M=3000, seed0=123)
print(res)

# 10000 samples
res = test_state_1_design_even_sector(n=6, M=10000, seed0=123)
print(res)


{'n': 6, 'samples': 300, 'trace(rhō)': 0.9999999999999992, 'weight_even': 0.9999999999999992, 'weight_odd': 0.0, 'fro_error_vs_even_max_mixed': 0.05330757292393401}
{'n': 6, 'samples': 3000, 'trace(rhō)': 0.9999999999999989, 'weight_even': 0.9999999999999989, 'weight_odd': 0.0, 'fro_error_vs_even_max_mixed': 0.0176278313148393}
{'n': 6, 'samples': 10000, 'trace(rhō)': 0.9999999999999991, 'weight_even': 0.9999999999999991, 'weight_odd': 0.0, 'fro_error_vs_even_max_mixed': 0.009929400887173414}


# 1-design on the odd/even parity sectors independently

In [33]:
import numpy as np, numpy.linalg as npl
from ff.ff_lib import random_FF_state
from ff import jordan_wigner_majoranas

def parity_projectors(n):
    Z = np.array([[1,0],[0,-1]], complex)
    P = np.array([[1]], complex)
    for _ in range(n): P = np.kron(P, Z)
    Id = np.eye(2**n, dtype=complex)
    return (Id+P)/2, (Id-P)/2

n, M = 4, 10000
P_even, P_odd = parity_projectors(n)
gam = jordan_wigner_majoranas(n)  # δ-normalized Majoranas

rho_bar = np.zeros((2**n, 2**n), complex)
rng = np.random.default_rng(0)

for t in range(M):
    psi = random_FF_state(n, pure=True, seed=int(rng.integers(1<<31)))  # even sector
    psi = np.asarray(psi).reshape(-1,1)
    # Flip to odd with a Majorana (keeps Gaussian & makes odd)
    psi = gam[0] @ psi
    psi /= npl.norm(psi)
    rho_bar += psi @ psi.conj().T

rho_bar /= M
target = P_odd / (2**(n-1))
print("‖ρ̄ - Π_odd/2^{n-1}‖_F =", npl.norm(rho_bar - target, 'fro'))
print("tr ρ̄ =", np.trace(rho_bar).real, " odd weight =", np.trace(P_odd@rho_bar).real)

‖ρ̄ - Π_odd/2^{n-1}‖_F = 0.008851491398124542
tr ρ̄ = 0.9999999999999988  odd weight = 0.9999999999999988
